In [297]:
from PIL import Image, ImageDraw
import numpy as np

In [311]:
# load images to PIL
face = Image.open("images/frontal.jpg")
mask = Image.open("images/mustache.png")
mask = mask.crop(mask.getbbox())
# create draw object for face image
draw = ImageDraw.Draw(face)
mask.size

(456, 107)

In [299]:
mask = np.array(mask)

In [300]:
mask = Image.fromarray(mask[:,:,:])


In [301]:
def find_coeffs(pa, pb):
    
    matrix = []
    for p1, p2 in zip(pa, pb):
        matrix.append([p1[0], p1[1], 1, 0, 0, 0, -p2[0]*p1[0], -p2[0]*p1[1]])
        matrix.append([0, 0, 0, p1[0], p1[1], 1, -p2[1]*p1[0], -p2[1]*p1[1]])

    A = np.matrix(matrix, dtype=np.float)
    B = np.array(pb).reshape(8)

    res = np.dot(np.linalg.inv(A.T * A) * A.T, B)
    return np.array(res).reshape(8)

def smallestBoxSize(coordinates):
    x, y = zip(*coordinates)
    return max(x)-min(x), max(y)-min(y)


In [302]:
lx, ly = mask.size
src = [(0,0), (0,ly), (lx, 0), (lx,ly)]
# dst = [(0,0), (0,ly*4), (lx,0), (lx,ly)]
# mapping = find_coeffs(dst, src)
# new_size = smallestBoxSize(dst)
# mask = mask.transform(new_size, Image.PERSPECTIVE, mapping)
# mask = mask.crop(mask.getbbox())

def transform4point(img, src, dst):
    lx, ly = img.size
    mapping = find_coeffs(dst, src)
    # new_size = smallestBoxSize(dst)
    img = img.transform((lx, ly), Image.PERSPECTIVE, mapping)
    img = img.crop(img.getbbox())
    lx, ly = img.size
    img = img.resize((lx*3,round(ly*1.3)))
    return img

In [303]:
from face_recognition import face_landmarks

In [304]:
modelsize = ["small", "large"][0]
landmarks = face_landmarks(np.array(face), model=modelsize)

i=0
def dst_quad(landmark, model="large"):
    if model == "large":
        a = landmark["top_lip"][1]
        b = landmark["top_lip"][5]
        c = landmark["bottom_lip"][5]
        d = landmark["bottom_lip"][1]
    elif model == "small":
        return 0,0,0,0
    else:
        raise ValueError("Cannot determine type of model; use \"small\" or \"large\"")
    return a,b,c,d
a,b,c,d = dst_quad(landmarks[0], model=modelsize)
# draw.line([a,b,c,d])
for land in landmarks:
    for name, points in land.items():
        draw.line(points, fill='red', width=2)
        for xy in points:
            draw.text(xy,f"{i}")
            if i in [49, 53, 65, 61]:
                print(i,xy,points)

            i+=1
face.show()


In [307]:
# get quadrilateral from triangle based on frontal face
def triangle_points(landmark, model="small"):
    left = landmark["left_eye"][0]
    right = landmark["right_eye"][0]
    if model=="small":
        bottom = landmark["nose_tip"][0]
    elif model=="large":
        bottom = landmark["nose_tip"][2]
    else:
        raise ValueError("Cannot determine type of model; use \"small\" or \"large\"")
    return left, right, bottom

triple = triangle_points(landmarks[0], model=modelsize)
triple

((166, 197), (297, 198), (231, 266))

In [336]:
# dx is de afstand van het buitenoog tot het punt
lx, ly = mask.size
r = lx / ly
k = - 69 / 65 
y0 = 69
x = -y0 * r / ( k * r - 1 )
y = y0 + k * x
dx, dy = 65-x, 69-y
(x/65, y/69), (x, y), (dx, dy)

((0.8189697805773185, 0.18103021942268144),
 (53.233035737525704, 12.49108514016502),
 (11.766964262474296, 56.50891485983498))

In [340]:
# sr is de ratio van buitenoog tot de neus
sk = (dx**2+y**2)**.5   # schuin klein
sg = (69**2+65**2)**.5  # schuin groot
sr = sk/sg  # schuin ratio
sr = 0.18

0.18103021942268147

In [341]:
landmarks

[{'nose_tip': [(231, 266)],
  'left_eye': [(166, 197), (203, 200)],
  'right_eye': [(297, 198), (259, 200)]}]

In [344]:
# ro is de ratio van het buitenoog naar het binnenoog
gem_oog = 38 # gemiddelde afstand tussen binnen en buiten oog
ro = dx/38 #
ro = 0.31

0.3096569542756394

In [ ]:
def midPoint(a, b, ratio)
    """
    returns the point in between a and b at a distance of ratio*(b-a)
    :param a: point to start the walk
    :param b: point to walk to
    :return: point between a and b, when you walked :ratio: part of total
    """
    ax, ay = a
    bx, by = b
    dx = (bx-ax)*ratio
    dy = (by-ay)*ratio
    return ax+dx, ay+dy

In [ ]:
def quadrilateral(landmark):
    """
    Find convenient quadrilateral from 5 landmark points for transformation of mask
    :param landmark: landmark points based on small or large model from face_recognition
    :return: 4 xy tuples with corner values of quadrilateral [top_left, top_right, botton_left, bottom_right]
    """
    lEyeOut, lEyeIn = landmark["left_eye"]
    rEyeOut, rEyeIn = landmark["right_eye"]
    noseTip = landmark["nose_tip"]
    if len(noseTip) == 1:
        noseTip = noseTip[0]
    elif len(noseTip) == 5:
        noseTip = noseTip[2]
    else:
        raise ValueError("Doesn't recognize landmarks; len(nose_tip) should be 1 or 5")
    
    topLeft     = midPoint(a=lEyeOut, b=lEyeIn,  ratio=0.31)
    topRight    = midPoint(a=rEyeOut, b=rEyeIn,  ratio=0.31)
    bottomLeft  = midPoint(a=lEyeOut, b=noseTip, ratio=0.18)
    bottomRight = midPoint(a=rEyeOut, b=noseTip, ratio=0.18)
    return topLeft, topRight, bottomLeft, bottomRight

In [317]:
def triangle_to_square(triple)
    


SyntaxError: invalid syntax (<ipython-input-317-a82cd07cf3b4>, line 1)

In [277]:
# face.show()


In [278]:
# lEyeOut, lEyeIn = landmarks[0]["left_eye"]
# rEyeOut, rEyeIn = landmarks[0]["right_eye"]
nose = landmarks[0]["nose_tip"][2]

In [279]:
dst = [a,c,b,d]

x,y = zip(*dst)
xmin,ymin = min(x), min(y)
dst = [(xi-xmin,yi-ymin) for xi,yi in dst]


In [280]:
mask = transform4point(img=mask,src=src,dst=dst)

In [281]:
lx, ly = mask.size

In [282]:
face.paste(
    mask,
    (nose[0] - lx//2, nose[1]),
    mask)

In [283]:
face.show()

(414, 702)